In [1]:
import pandas as pd
import numpy as np
import psycopg2
from utils.db_connect import create_db_connection
# from sqlalchemy import create_engine

In [2]:
total_seats = 90

In [3]:
conn = create_db_connection(host='localhost')

In [4]:
# count tickets for each trip
conn.rollback()
query = """
SELECT 
    COUNT(fare) as passengers_in,
    trip_id,
    MAX(school) as school,
    MAX(hospital) as hospital,
    MAX(peak_hour) as peak_hour,
    DATE_TRUNC('day', timestamp) as day,
    MAX(timestamp) as timestamp
FROM 
    raw_tickets 
GROUP BY 
    trip_id, 
    DATE_TRUNC('day', timestamp),
    DATE_TRUNC('month', timestamp),
    DATE_TRUNC('year', timestamp)
ORDER BY 
    day
"""

tickets = pd.read_sql(query, conn)
tickets.head()


,passengers_in,trip_id,school,hospital,peak_hour,day,timestamp
0,485,0004245912025030620250612,0,0,1,2025-05-19,2025-05-19 14:31:00
1,379,0004229872024090920250612,0,0,1,2025-05-19,2025-05-19 18:51:00
2,512,0004246222025030620250612,0,0,1,2025-05-19,2025-05-19 08:12:00
3,238,0004211222024090920250612,0,0,1,2025-05-19,2025-05-19 18:45:00
4,160,0004229912024090920250612,0,0,0,2025-05-19,2025-05-19 06:23:00


In [5]:
# count people out for each trip
conn.rollback()
query = """
SELECT 
    COUNT(status) as passengers_out,
    trip_id,
    DATE_TRUNC('day', timestamp) as day,
    MAX(timestamp) as timestamp
FROM 
    raw_sensors
GROUP BY 
    trip_id, 
    DATE_TRUNC('day', timestamp),
    DATE_TRUNC('month', timestamp),
    DATE_TRUNC('year', timestamp)
ORDER BY 
    day
"""

sensors =pd.read_sql(query, conn)
sensors.head()

,passengers_out,trip_id,day,timestamp
0,186,0004245992025030620250612,2025-05-19,2025-05-19 20:55:00
1,100,0004229512024090920250612,2025-05-19,2025-05-19 13:14:00
2,71,0004211342024090920250612,2025-05-19,2025-05-19 16:25:00
3,130,0004245912025030620250612,2025-05-19,2025-05-19 14:31:00
4,173,0004229872024090920250612,2025-05-19,2025-05-19 18:51:00


In [6]:
conn.rollback()

query = """
SELECT *
FROM trips
"""
trips = pd.read_sql(query, conn)
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
0,607,0000001432024090920250612,0004187192024090920250612,Sorni,False,None,NaN
1,607,0000001442024090920250612,0004187202024090920250612,Pressano Piazza,False,None,NaN
2,607,0000001442024090920250612,0004187212024090920250612,Pressano Piazza,False,None,NaN
3,613,0000001482024090920250612,0004187222024090920250612,Pressano Piazza,False,None,NaN
4,610,0000001462024090920250612,0004187232024090920250612,Lavis Stazione Fs,False,None,NaN


In [7]:
conn.rollback()

query = """
select trip_id, timestamp, traffic_level, normal, traffic
from traffic
"""
traffic = pd.read_sql(query, conn)
traffic.head()

,trip_id,timestamp,traffic_level,normal,traffic
0,0004188952024090920250612,2025-05-19 09:37:00,no traffic/low,778,777
1,0004190132024090920250612,2025-05-19 07:04:00,no traffic/low,905,903
2,0004190322024090920250612,2025-05-19 07:38:00,no traffic/low,777,776
3,0004190342024090920250612,2025-05-19 07:42:00,no traffic/low,745,764
4,0004193602024090920250612,2025-05-19 06:46:00,no traffic/low,909,910


In [8]:
conn.rollback()

query = """
select *
from events
where latitude is not null
"""

events = pd.read_sql(query, conn)
events = events.drop_duplicates(subset=['day_event'])

In [9]:
conn.rollback()

query = """
select *
from weather
"""
weather = pd.read_sql(query, conn)
weather = weather.rename(columns={'timestamp':'time'})
weather.head()

,measurement_id,time,latitude,longitude,weather_code,precipitation_probability,temperature,hour
0,169,2025-05-19 20:17:00,46.04,11.119999,1,0.0,10.5,2025-05-18 00:00:00
1,170,2025-05-19 20:17:00,46.04,11.119999,1,0.0,9.7,2025-05-18 01:00:00
2,171,2025-05-19 20:17:00,46.04,11.119999,1,0.0,9.1,2025-05-18 02:00:00
3,172,2025-05-19 20:17:00,46.04,11.119999,1,0.0,8.7,2025-05-18 03:00:00
4,173,2025-05-19 20:17:00,46.04,11.119999,1,0.0,8.6,2025-05-18 04:00:00


In [10]:
print(tickets.shape)

(376, 7)


In [11]:
print(sensors.shape)

(379, 4)


In [12]:
len(sensors['trip_id'].unique())

379

In [13]:
len(tickets['trip_id'].unique())

376

In [14]:
trips.shape

(3613, 7)

In [15]:
merged_tickets = tickets.merge(sensors, on=['trip_id','timestamp']).merge(trips, on='trip_id')#.sort_values('trip_id')
merged_tickets.head()

,passengers_in,trip_id,school,hospital,peak_hour,day_x,timestamp,passengers_out,day_y,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
0,485,0004245912025030620250612,0,0,1,2025-05-19,2025-05-19 14:31:00,130,2025-05-19,404,0000000532025030620250612,Guardini Centochiavi,True,D670_T0892_Ritorno_PRINCIPALE_ALTA_sub1,1.0
1,379,0004229872024090920250612,0,0,1,2025-05-19,2025-05-19 18:51:00,173,2025-05-19,594,0000001712024090920250612,Corso Verona Ospedale,True,D461_R5514a_Ritorno_PRINCIPALE_sub2,1.0
2,512,0004246222025030620250612,0,0,1,2025-05-19,2025-05-19 08:12:00,187,2025-05-19,404,0000000022025030620250612,Guardini Centochiavi,True,D669_T0892a_Ritorno_PRINCIPALE_BASSA_sub2,1.0
3,238,0004211222024090920250612,0,0,1,2025-05-19,2025-05-19 18:45:00,105,2025-05-19,400,0000000102024090920250612,Piazza Dante Dogana,True,D602_T0522k_Ritorno_sub1,1.0
4,160,0004229912024090920250612,0,0,0,2025-05-19,2025-05-19 06:23:00,52,2025-05-19,594,0000000672024090920250612,Isera P.Zza S.Vincenzo-O.,True,D464_R5520b_Ritorno_sub1,1.0


In [16]:
merged_tickets.shape

(376, 15)

merge traffic with trips to get shape id for later merge

In [17]:
traffic_merged = traffic.merge(trips, on='trip_id')
traffic_merged.head()

,trip_id,timestamp,traffic_level,normal,traffic,route_id,service_id,trip_headsign,direction_id,shape_id,wheelchair_accessible
0,0004188952024090920250612,2025-05-19 09:37:00,no traffic/low,778,777,400,0000000102024090920250612,Piazza Dante Dogana,True,D170_F0512_Ritorno_sub1,1.0
1,0004190132024090920250612,2025-05-19 07:04:00,no traffic/low,905,903,404,0000000032024090920250612,Verona Palazzo Stella,False,D660_T0865_Andata_sub1,1.0
2,0004190322024090920250612,2025-05-19 07:38:00,no traffic/low,777,776,400,0000000102024090920250612,Povo Polo Sociale Cap.,False,D599_T0521h_Andata_sub1,1.0
3,0004190342024090920250612,2025-05-19 07:42:00,no traffic/low,745,764,404,0000000032024090920250612,Piazza Dante Dogana,True,D179_F0842b_Ritorno_ALTA_sub1,1.0
4,0004193602024090920250612,2025-05-19 06:46:00,no traffic/low,909,910,400,0000000022024090920250612,Oltrecastello,False,D593_T0501b_Andata_PRINCIPALE_sub2,1.0


In [18]:
len(traffic_merged['shape_id'].unique())

27

In [19]:
traffic_merged.shape

(27, 11)

Before aggregating with traffic we need to make sure that the shape_id is included inside the table so to do we need to join the table with the trips table on the trip_id column to get the shape_id column.

In [20]:
traffic_tickets = merged_tickets.merge(traffic_merged, on=['shape_id'], how='right')[['traffic_level', 'normal', 'traffic', 'passengers_in', 'trip_id_x', 'school', 'hospital', 'peak_hour', 'timestamp_x', 'passengers_out']]
traffic_tickets.head()

,traffic_level,normal,traffic,passengers_in,trip_id_x,school,hospital,peak_hour,timestamp_x,passengers_out
0,no traffic/low,778,777,292.0,0004210802024090920250612,0.0,0.0,1.0,2025-05-19 21:25:00,93.0
1,no traffic/low,778,777,304.0,0004212802024090920250612,0.0,0.0,1.0,2025-05-19 16:45:00,88.0
2,no traffic/low,778,777,169.0,0004206532024090920250612,0.0,0.0,1.0,2025-05-19 09:25:00,78.0
3,no traffic/low,778,777,229.0,0004212842024090920250612,0.0,0.0,1.0,2025-05-19 20:45:00,133.0
4,no traffic/low,778,777,290.0,0004210812024090920250612,0.0,0.0,0.0,2025-05-19 22:25:00,84.0


In [21]:

print(traffic_tickets.shape)

(267, 10)


In [22]:
traffic_tickets['timestamp_x'] = pd.to_datetime(traffic_tickets['timestamp_x'])
weather['hour'] = pd.to_datetime(weather['hour'])

traffic_tickets = traffic_tickets.sort_values('timestamp_x')
weather = weather.sort_values('hour')

In [23]:
traffic_tickets = traffic_tickets.dropna(axis=0, subset=['timestamp_x'])

In [24]:
merged = pd.merge_asof(
    traffic_tickets,
    weather[['hour', 'temperature', 'precipitation_probability', 'weather_code', 'latitude', 'longitude']],
    left_on='timestamp_x',
    right_on='hour',          # match on timestamp column
    direction='backward',     # can also use 'backward' or 'forward'
    tolerance=pd.Timedelta('1000h')  # optional: max time difference allowed
)

In [25]:
merged.shape

(265, 16)

In [26]:
merged['timestamp_x']

0     2025-05-19 05:56:00
1     2025-05-19 05:57:00
2     2025-05-19 06:15:00
3     2025-05-19 06:25:00
4     2025-05-19 06:42:00
              ...        
260   2025-05-19 23:25:00
261   2025-05-19 23:25:00
262   2025-05-19 23:29:00
263   2025-05-19 23:29:00
264   2025-05-19 23:59:00
Name: timestamp_x, Length: 265, dtype: datetime64[ns]

In [27]:
merged.columns

Index(['traffic_level', 'normal', 'traffic', 'passengers_in', 'trip_id_x',
       'school', 'hospital', 'peak_hour', 'timestamp_x', 'passengers_out',
       'hour', 'temperature', 'precipitation_probability', 'weather_code',
       'latitude', 'longitude'],
      dtype='object')

In [28]:
events['day_event'] = pd.to_datetime(events['day_event'])

In [29]:
events.head()

,event_id,event_name,day_event,start_time,end_time,location_event,latitude,longitude
0,200,"DANTE VIBES: PAROLE, IMMAGINI, GIOCHI INTORNO ...",2025-04-28,08:30:00,19:30:00,"Biblioteca comunale sede centrale, via Roma 55...",46.069905,11.120876
1,201,"DANTE VIBES: PAROLE, IMMAGINI, GIOCHI INTORNO ...",2025-04-29,08:30:00,19:30:00,"Biblioteca comunale sede centrale, via Roma 55...",46.069905,11.120876
2,202,"DANTE VIBES: PAROLE, IMMAGINI, GIOCHI INTORNO ...",2025-04-30,08:30:00,19:30:00,"Biblioteca comunale sede centrale, via Roma 55...",46.069905,11.120876
3,203,"DANTE VIBES: PAROLE, IMMAGINI, GIOCHI INTORNO ...",2025-05-01,08:30:00,19:30:00,"Biblioteca comunale sede centrale, via Roma 55...",46.069905,11.120876
4,204,"DANTE VIBES: PAROLE, IMMAGINI, GIOCHI INTORNO ...",2025-05-02,08:30:00,19:30:00,"Biblioteca comunale sede centrale, via Roma 55...",46.069905,11.120876


In [30]:
# Ensure both columns are in datetime format (date only, no time)
merged['timestamp'] = merged['timestamp_x']
events['day_event'] = pd.to_datetime(events['day_event']).dt.date
merged['timestamp_x'] = pd.to_datetime(merged['timestamp_x']).dt.date  # or 'date' depending on column name

# Merge on the date
final_with_events = pd.merge(events, merged, left_on='day_event', right_on='timestamp_x', how='right')  # use 'left', 'right', or 'outer' if needed

In [31]:
final_with_events

,event_id,event_name,day_event,start_time,end_time,location_event,latitude_x,longitude_x,traffic_level,normal,...,peak_hour,timestamp_x,passengers_out,hour,temperature,precipitation_probability,weather_code,latitude_y,longitude_y,timestamp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,782,...,0.0,2025-05-19,174.0,2025-05-19 05:00:00,12.0,0.0,0,46.04,11.119999,2025-05-19 05:56:00
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,909,...,0.0,2025-05-19,106.0,2025-05-19 05:00:00,12.0,0.0,0,46.04,11.119999,2025-05-19 05:57:00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,778,...,0.0,2025-05-19,112.0,2025-05-19 06:00:00,13.9,0.0,0,46.04,11.119999,2025-05-19 06:15:00
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,809,...,0.0,2025-05-19,180.0,2025-05-19 06:00:00,13.9,0.0,0,46.04,11.119999,2025-05-19 06:25:00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,713,...,0.0,2025-05-19,98.0,2025-05-19 06:00:00,13.9,0.0,0,46.04,11.119999,2025-05-19 06:42:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,778,...,0.0,2025-05-19,78.0,2025-05-19 23:00:00,14.1,0.0,3,46.04,11.119999,2025-05-19 23:25:00
261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,778,...,0.0,2025-05-19,78.0,2025-05-19 23:00:00,14.1,0.0,3,46.04,11.119999,2025-05-19 23:25:00
262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,809,...,0.0,2025-05-19,140.0,2025-05-19 23:00:00,14.1,0.0,3,46.04,11.119999,2025-05-19 23:29:00
263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no traffic/low,809,...,0.0,2025-05-19,140.0,2025-05-19 23:00:00,14.1,0.0,3,46.04,11.119999,2025-05-19 23:29:00


In [32]:
final_with_events['event_dummy'] = final_with_events['day_event'].notna().astype(int)

In [33]:
selected_features = ['trip_id_x', 'peak_hour', 'timestamp', 'temperature', 'precipitation_probability', 'weather_code', 'normal', 'traffic', 'traffic_level', 'event_dummy', 'passengers_in', 'passengers_out']

In [34]:
final_with_events['total_capacity'] = total_seats
final_with_events['congestion_rate'] = (final_with_events['passengers_in'] - final_with_events['passengers_out']) / final_with_events['total_capacity']

In [35]:
final_with_events['seconds_from_midnight'] = (
    final_with_events['timestamp'].dt.hour * 3600 +
    final_with_events['timestamp'].dt.minute * 60 +
    final_with_events['timestamp'].dt.second
)

In [36]:
selected_features = ['trip_id_x', 'peak_hour', 'timestamp', 'seconds_from_midnight', 'temperature', 'precipitation_probability', 'weather_code', 'normal', 'traffic', 'traffic_level', 'event_dummy', 'passengers_in', 'passengers_out', 'total_capacity', 'congestion_rate']

In [37]:
final_with_events[selected_features][final_with_events['congestion_rate'] >= 0]

,trip_id_x,peak_hour,timestamp,seconds_from_midnight,temperature,precipitation_probability,weather_code,normal,traffic,traffic_level,event_dummy,passengers_in,passengers_out,total_capacity,congestion_rate
0,0004209392024090920250612,0.0,2025-05-19 05:56:00,21360,12.0,0.0,0,782,798,no traffic/low,0,496.0,174.0,90,3.577778
1,0004211192024090920250612,0.0,2025-05-19 05:57:00,21420,12.0,0.0,0,909,910,no traffic/low,0,338.0,106.0,90,2.577778
2,0004237172025010120250612,0.0,2025-05-19 06:15:00,22500,13.9,0.0,0,778,777,no traffic/low,0,290.0,112.0,90,1.977778
3,0004197522024090920250612,0.0,2025-05-19 06:25:00,23100,13.9,0.0,0,809,836,no traffic/low,0,518.0,180.0,90,3.755556
4,0004220122024090920250612,0.0,2025-05-19 06:42:00,24120,13.9,0.0,0,713,764,no traffic/low,0,328.0,98.0,90,2.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,0004210822024090920250612,0.0,2025-05-19 23:25:00,84300,14.1,0.0,3,778,777,no traffic/low,0,198.0,78.0,90,1.333333
261,0004209702024090920250612,0.0,2025-05-19 23:25:00,84300,14.1,0.0,3,778,777,no traffic/low,0,198.0,78.0,90,1.333333
262,0004243882025030620250612,0.0,2025-05-19 23:29:00,84540,14.1,0.0,3,809,779,no traffic/low,0,332.0,140.0,90,2.133333
263,0004243922025030620250612,0.0,2025-05-19 23:29:00,84540,14.1,0.0,3,809,779,no traffic/low,0,332.0,140.0,90,2.133333


In [38]:
final_with_events[selected_features].groupby('peak_hour')[['passengers_in', 'passengers_out']].agg('mean')

,passengers_in,passengers_out
peak_hour,,
0.0,277.406250,97.406250
1.0,321.716418,118.412935


# Model

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [40]:
encoder_traffic = LabelEncoder()
final_with_events['encoded_traffic'] = encoder_traffic.fit_transform(final_with_events['traffic_level'])


In [41]:
features = ['peak_hour', 'seconds_from_midnight', 'temperature', 'precipitation_probability', 'weather_code', 'encoded_traffic', 'event_dummy', 'congestion_rate']

In [42]:
X = final_with_events[features].drop('congestion_rate', axis=1)
y = final_with_events[features]['congestion_rate']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True)

In [44]:
rf = RandomForestRegressor()

# model.fit(X_train, y_train)


In [45]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators' : [100, 200, 300],
    'criterion' : ['squared_error', 'friedman_mse'],
    'max_depth' : [10, 100, 500, 1000],
    'min_samples_split' : [2, 5, 10],
    'max_features' : ['sqrt', 'log2']
}

grid = GridSearchCV(rf, param_grid, cv=5)

grid.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
grid.best_params_

{'criterion': 'friedman_mse',
 'max_depth': 10,
 'max_features': 'log2',
 'min_samples_split': 10,
 'n_estimators': 100}

In [ ]:
best_rf = grid.best_estimator_
# best_rf.predict(X_test)

array([0.85969705, 3.0330249 , 3.17316572, 3.2888994 , 1.66484593,
       2.47200395, 3.11601366, 2.70521345, 0.60169056, 1.49397714,
       3.29938833, 3.51815571, 2.1728817 , 1.66025974, 1.53695252,
       2.49584445, 2.84139565, 2.53244695, 2.68337448, 3.29938833,
       2.39304682, 3.17385015, 1.63213852, 1.60265365, 3.18749275,
       0.53043912, 1.56445312, 3.03388717, 1.0716376 , 3.05996785,
       2.64192587, 2.87608164, 2.67131648, 3.2888994 , 1.66025974,
       2.67131648, 0.67116879, 1.98921642, 2.26148104, 3.01842661,
       2.42002697, 2.85385937, 0.53043912, 0.9952914 , 1.96469662,
       1.50975444, 2.53244695, 1.66025974, 2.73015655, 1.69723006,
       0.62932421, 2.84763035, 2.1728817 , 2.96540353, 3.12403629,
       1.48965165, 2.64152262, 2.70521345, 1.0963296 , 1.70493264,
       1.70084263, 3.19604337, 2.07551977, 3.15877441, 1.56990893,
       3.069343  , 1.01961504, 1.30033117, 2.21680481, 1.81630176,
       2.76829278, 3.20183719, 2.5811873 , 0.60169056, 2.84763

In [ ]:
best_rf.score(X_test, y_test)

0.6049094134182373